# Códigos Tidene

## Leitura dos textos

### Opção 1: Leitura de corpus (textos) de tamanho muito grande

Classe readCorpus - Permite a extração de colunas específicas.
Requisitos: O arquivo csv deve ter uma linha de cabeçalho, que nomeia cada um dos colunas (campos)

Parâmetros de entrada:
   - csvfile => nome do arquivo csv
   - list_of_fields_to_read=[] ==> lista de colunas que deverão ser lidas (se não colocar valor, ele assume que deverá ler os valores de todas as colunas)
   - tokenizer = None => recebe um objeto do tipo tokenizador (caso tenha valor, retornará o texto já tokenizado utilizando aquele tokenizador) == vale apenas para lista de campos = 1
   - encoding => padrão de codificação (default = utf8)

Saída: iterador que percorre cada linha do corpus


#### Exemplo de entrada .csv

#### subgroup,maingroup,subclas,clas,section,othersipcs,data
B03B00402,B03B004,B03B,B03,B,B07B00408,separation apparatus this invention relates to a method for separation of a light material from a heavier material a separation table of vibrator type and a cyclone and a

B03B00500,B03B005,B03B,B03,B,B01D01102-E02Fn means00388,method and installation for desalinating sand and suction hopper comprising such an installation the invention 


In [ ]:
import csv

class readCorpus(object):
    def __init__(self,csvfile,list_of_fields_to_read=[],tokenizer=None,encoding='utf8'):
        self.csvfile = csvfile
        self.fields = list_of_fields_to_read
        self.tokenizer = tokenizer
        self.encoding = encoding
    
    def __iter__(self):
        f = open(self.csvfile,encoding=self.encoding, errors='ignore')
        reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_MINIMAL) #separador dos campos\n",
        headers = next(reader, None)
        if (len(self.fields) <= 0):
            self.fields = headers
        selected_field_indexes = []
        for idx,field in enumerate(headers):
            if field in self.fields:
                selected_field_indexes.append(idx)

        for line in reader:
            if line:
                yield [line[idx] for idx in selected_field_indexes] if (len(selected_field_indexes)>1) else (line[selected_field_indexes[0]] if not self.tokenizer else tokenizer.tokenize(line[selected_field_indexes[0]]))
                        

#### Exemplo de uso

In [ ]:
corpus = readCorpus("data/train_min.csv",list_of_fields_to_read=['sentiment','review'])
textos = [texto for texto in corpus]
print(textos[3])

In [ ]:
#se quiser armazenar em uma estrutura do tipo DataFrame
import pandas as pd
df_textos = pd.DataFrame(textos,columns=['sentiment','review']) # armazenando somente os textos

In [ ]:
df_textos

### Opção 2: Ler direto do arquivo .csv em uma estrutura tipo DataFrame

In [ ]:
import pandas as pd
df_textos = pd.read_csv('data/train.csv',encoding='utf8')['review']

In [ ]:
print(df_textos)

## Limpeza dos textos + redução de dimensionalidade

In [ ]:
import nltk
import numpy as np

### Tokenização

In [ ]:
import nltk
from nltk.tokenize import *

# instancia o tokenizador
tokenizer=nltk.tokenize.RegexpTokenizer("[a-zA-Z']+")

# ... este, por exemplo, separa por palvras e deixa as que tem ' juntas 
# exemplo de uso
tokenizer.tokenize("my can't go should't 321")


In [ ]:
corpus = readCorpus("data/train_min.csv",list_of_fields_to_read=['review'],tokenizer=tokenizer)
tokens = [texto for texto in corpus]   #values.tolist()
print(tokens[1])


### Remoção de stopwords

In [ ]:
from nltk import download
from nltk.corpus import stopwords
download('stopwords')

#### Lista de stopwords disponível na nltk

In [ ]:
stop_words = stopwords.words('english')
print(stop_words)


#### Remove stopwords

In [ ]:
tokens_noStp = [[word for word in texto if not word in stop_words and len(word) > 1] for texto in tokens]
print(tokens_noStp[1])

#### Remoção de radicais (utilizando lemmatizador ou stemmer)

(https://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization)

In [ ]:
# Lematizador
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
tokens_lem = [[wordnet_lemmatizer.lemmatize(word) for word in texto] for texto in tokens_noStp]
print(tokens_lem[1])

In [ ]:
# Stemmer
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
tokens_stem = [[porter_stemmer.stem(word) for word in texto] for texto in tokens_noStp]
print(tokens_stem[1])

### Salvando no disco o corpus serializado em forma de (indice,frequencia)

$conda install gensim


In [ ]:
import gensim


In [ ]:
# primeiro, monta-se o dicionario (em forma de indice, palavra unica)
dictionary = gensim.corpora.Dictionary(tokens_lem)
dictionary.save('dictionary.dict')
print(dictionary.token2id)

#### Representação Bag-of-Words (contagem de palavras)

In [ ]:
dictionary = gensim.corpora.Dictionary.load("dictionary.dict") #carrega o dicionario do disco

bowcorpus = [dictionary.doc2bow(texto) for texto in tokens_lem] #vetoriza para representacao (indice,freq)
gensim.corpora.MmCorpus.serialize('bowcorpus.mm', bowcorpus)  # grava no disco
print(bowcorpus[1])

#### Representação Tf-idf (https://radimrehurek.com/gensim/tutorial.html)

In [ ]:
bowcorpus = gensim.corpora.MmCorpus('bowcorpus.mm')
tfidf_vectorizer = gensim.models.TfidfModel(bowcorpus)
tfidf_corpus_matrix = tfidf_vectorizer[bowcorpus]
gensim.corpora.MmCorpus.serialize('tfidf_corpus_matrix.mm', tfidf_corpus_matrix)  # grava no disco

print(tfidf_corpus_matrix[1])
